In [ ]:
#|default_exp export

In [ ]:
#|export
from __future__ import annotations

In [ ]:
#|export
from nbdev.process import *
from nbdev.imports import *
from nbdev.maker import *

In [ ]:
_test_file = "00_core.ipynb"

In [ ]:
#|export
from collections import defaultdict
from fastcore.foundation import L, ifnone
from execnb.nbio import *

class ExportTestProc:
    "A test proc that watches for `#|default_exp` and `#|test`"
    def __init__(self): self.tests = defaultdict(L)
    def _default_exp_(self, cell, exp_to): self.default_exp = f'test_{exp_to}'
    def _test_(self, cell, exp_to=None, nm=None, tst_cls=None): self.tests[self.default_exp].append(cell)

In [ ]:
#|export
_re_test = re.compile(r'#\|\s*test\s*$', re.MULTILINE)
_re_import = re.compile(r'#\|\s*test\s*import\s*$', re.MULTILINE)
_tab = "    "

In [ ]:
#|export
def get_directive(cell, key, default=None): 
    "Extract a top level directive from `cell`"
    return cell.directives_.get(key, default)

def _is_test_cell(cell): return cell.cell_type == "code" and get_directive(cell,'test')

In [ ]:
def _mark_test(s):
    ft = exec_new("import fastcore.test as ft")["ft"].__all__
    kinds = [(o,f'ft.{o}') for o in ft if o.startswith("test_")]
    for k,v in kinds: s = s.replace(k,v)
    return s

In [ ]:
#|export
def convert_pytest(cell, unittest=False):
    "Wraps cell contents into a pytest function"
    directive = get_directive(cell, "test")
    if _is_test_cell(cell):
        if "import" not in directive and "case" not in directive:
            content = '\n'.join([f"{_tab}{c}" for c in cell.source.split("\n")])
            content = _mark_test(content)
            if unittest: cell.source = f'def test_{directive[0]}(self):\n{content}'
            else: cell.source = f'def test_{directive[0]}():\n{content}'
        else:
            cell.source = cell.source.replace("from fastcore.test import *", "import fastcore.test as ft")

In [ ]:
#|export
_re_defaultexp = re.compile(r'^\s*#\|\s*default_exp\s+(\S+)', flags=re.MULTILINE)
def _default_exp(nb):
    "get the default_exp from a notebook"
    code_src = nb.cells.filter(lambda x: x.cell_type == 'code').attrgot('source')
    default_exp = first(code_src.filter().map(_re_defaultexp.search).filter())
    return default_exp.group(1) if default_exp else None
    

In [ ]:
#|export
def construct_imports(nb, unittest=False):
    "Generates the test imports for the notebook"
    libname = get_config().lib_name
    exp = _default_exp(nb)
    imports = ['#| test import\n', f'from {libname}.{exp} import *\n']
    if unittest: imports += ['import unittest']
    nb.cells.insert(1, mk_cell(imports))

In [ ]:
#|export
def create_test_modules(path, dest, debug=False, mod_maker=ModuleMaker, unittest=False):
    "Creates test files from path, optionally with unittest support"
    exp = ExportTestProc()
    procs = [exp, functools.partial(convert_pytest, unittest=unittest)]

    if unittest: procs.append(convert_unittest)

    class InsertWarning(Processor):
        content = functools.partial(construct_imports, unittest=unittest)
        def begin(self): self.nb.cells.insert(1, mk_cell(self.content, 'markdown'))
    
    nb = NBProcessor(path, procs)
    insert_warning = InsertWarning(nb)
    nb.preprocs = insert_warning
    nb.process()
    for i,(mod,cells) in enumerate(exp.tests.items()):
        mm = mod_maker(dest=dest, name=exp.default_exp, nb_path=path, is_new=i==0, parse=False)
        mm.make(cells)

In [ ]:

#|eval: false
create_test_modules(_test_file, "tests")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
!pytest tests/